In [ ]:
!pip install hmeasure

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.utils import shuffle

# Generate synthetic data with unbalanced classes
n_samples = 200
X = np.ones(n_samples)
y = np.hstack([np.zeros(int(n_samples * 0.3)), np.ones(int(n_samples * 0.7))])

# Shuffle the data to randomly distribute the class labels
X, y = shuffle(X, y, random_state=42)

# K-Fold Cross-Validation
kf = KFold(n_splits=5)
kf_splits = list(kf.split(X, y))

# Time Series Cross-Validation
tscv = TimeSeriesSplit(n_splits=5)
tscv_splits = list(tscv.split(X))

# Plotting function
def plot_cv_indices(cv, X, y, ax, n_splits, title, lw=10):
    cmap_data = plt.cm.Paired
    for ii, (train, test) in enumerate(cv):
        indices = np.arange(len(X))
        ax.scatter(indices[train], [ii + 0.5] * len(train), c=cmap_data(0), marker='_', lw=lw)
        ax.scatter(indices[test], [ii + 0.5] * len(test), c=cmap_data(1), marker='_', lw=lw)
    ax.scatter(indices, [ii + 1.5] * len(X), c=cmap_data(y), marker='|', lw=lw)
    ax.set(ylim=[n_splits + 1.5, -0.2], xlim=[0, len(X)], xlabel='Sample index', ylabel='CV iteration',
           yticks=np.arange(n_splits + 1) + 0.5, yticklabels=list(range(n_splits)) + ['class'])
    ax.legend(['Training set', 'Testing set'], loc='upper right')
    ax.set_title(title, fontsize=15)

# Plotting the cross-validation splits
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

plot_cv_indices(kf_splits, X, y, axes[0], len(kf_splits), 'K-Fold Cross-Validation (k=5)')
plot_cv_indices(tscv_splits, X, y, axes[1], len(tscv_splits), 'Time Series Cross-Validation (5 Splits)')

# Adding overall plot title and captions
fig.suptitle('Comparison of K-Fold and Time Series Cross-Validation', fontsize=20)
axes[0].set_ylabel('CV Iteration and Class Labels', fontsize=12)
axes[0].set_xlabel('Sample Index', fontsize=12)
axes[1].set_ylabel('CV Iteration and Class Labels', fontsize=12)
axes[1].set_xlabel('Sample Index', fontsize=12)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig('CV_Compare.png', dpi=300)


In [ ]:
import pandas as pd
import numpy as np
# A Lite/fast/popular gradient boosting tree
import lightgbm as lgb
from sklearn import preprocessing
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

from hmeasure import h_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve, auc
from sklearn.model_selection import TimeSeriesSplit
from sklearn.calibration import calibration_curve

In [ ]:
def plot_calibration_curve(y_true, y_prob, ax, label, n_bins=100):
    """
    Plot the calibration curve for a classifier on given axes.

    Args:
    y_true: array-like of shape (n_samples,), True binary labels.
    y_prob: array-like of shape (n_samples,), Target scores, can either be probability estimates of the positive class.
    ax: matplotlib.axes.Axes, the axes object to plot on.
    label: str, label for the plot curve.
    n_bins: int, Number of probability bins to use.
    """
    # Calculate the calibration curve
    prob_true, prob_pred = calibration_curve(y_true, y_prob, n_bins=n_bins, strategy='uniform')

    # Plot the calibration curve on the given axes
    ax.plot(prob_pred, prob_true, marker='o', linewidth=1, label=label)

    # Plot the perfectly calibrated line if it's the first fold
    if label == 'Fold 1':
        ax.plot([0, 1], [0, 1], linestyle='--', color='red', label='Perfectly calibrated')

    ax.set_xlabel('Predicted probability')
    ax.set_ylabel('Actual probability')
    ax.set_title('Calibration Plot')
    ax.legend()
    ax.grid(True)

## Set up the GEV-GBDT

In [ ]:
# Define customized loss function

def GEV_Loss(y_pred, y_true, tau):
    y = y_true.get_label()
    x = y_pred
    t = tau
    grad = (((1-y)*np.exp(-(t*x+1)**(-1/t)))*((t*x+1)**(-1/t-1)))/(1-np.exp(-(t*x+1)**(-1/t))) - y*((t*x+1)**(-1/t-1))
    hess = (1/t+1)*t*y*((t*x+1)**(-1/t-2)) + (1-y)*( (np.exp(-(t*x+1)**(-1/t))*((t*x+1)**(-2/t-2)))/(1-np.exp(-(t*x+1)**(-1/t)))+
                                                  (np.exp(-2*((t*x+1)**(-1/t)))*((t*x+1)**(-2/t-2)))/((1-np.exp(-(t*x+1)**(-1/t)))**2)-
                                                ((1/t+1)*t*np.exp(-(t*x+1)**(-1/t))*((t*x+1)**(-1/t-2)))/(1-np.exp(-(t*x+1)**(-1/t))) )
    # grad : first-order partial derivative of y_pred
    # hess : second - order partial derivative of y_pred
    return grad, hess

def Gumbel(y_pred, y_true):
    # When tau is set to 0, the GEV function follows gumbel distribution
    y = y_true.get_label()
    x = y_pred
    grad = (np.exp(x)*(np.exp(np.exp(x))*y-1))/(np.exp(np.exp(x))-1)
    hess = np.exp(x)*y - ((np.exp(x-np.exp(x))*(1-np.exp(x)))/(1-np.exp(-np.exp(x)))-
                          (np.exp(2*x-2*np.exp(x)))/((1-np.exp(-np.exp(x)))**2))*(1-y)

    return grad, hess

def custom_loss(y_pred, y_true):

    if tau == 0:
        return Gumbel(y_pred = y_pred, y_true = y_true)
    else:
        return GEV_Loss(y_pred = y_pred, y_true = y_true, tau = tau)

In [ ]:
from scipy.stats import genextreme as gev

def transform_gev(data, tau):
    """
    Transform a 1D array with the Generalized Extreme Value (GEV) distribution.

    Parameters:
    - data: ndarray, the 1D array of data points to transform.
    - tau: float, the shape parameter of the GEV distribution.

    Returns:
    - transformed_data: ndarray, the transformed data using the GEV CDF.
    """
    # Estimate location and scale parameters from the data
    # Here, we use the provided tau (shape parameter) directly
    c, loc, scale = tau, np.mean(data), np.std(data)

    # Transform data using the GEV CDF with estimated parameters
    transformed_data = gev.cdf(data, c, loc=loc, scale=scale)

    return transformed_data

### An Example of Using the GEV-GBDT

In [ ]:
# Set the value of tau
tau = -0.35

# Set the hyperparameter of LightGBM
# Remember to set objective as our customised loss function
params = {"objective":"binary", "boosting_type":"gbdt", "verbose": -1, "metric": "auc", 'learning_rate': 0.01, 'max_depth': -1,  "objective": custom_loss}

# Assume we have X_train, y_train, X_test, y_test
lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_test, y_test, reference = lgb_train)

# Fit the training data
gbm = lgb.train(params,
                lgb_train,
                num_boost_round = 1000)

# Predict on the test set and output probability
probs = gbm.predict(X_test)
# Use link function to transform the raw output within 0 and 1
probs = transform_gev(probs, tau)

## LC Dataset

In [ ]:
def evaluate_model(model, X, y, loss_col_name, fig_name):
    tscv = TimeSeriesSplit(n_splits=5)
    fig, ax = plt.subplots(figsize=(12,8))
    auc_scores = []
    ks_stats = []
    h_measures = []
    cm_sum = np.array([[0, 0], [0, 0]]) # Initialize a confusion matrix sum
    actual_losses = []
    predicted_losses = []
    predicted_losses_in = []
    fold = 1
    for train_index, test_index in tscv.split(X):
        print('Fold trial starts...')
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train, y_train)
        probs = model.predict_proba(X_test)[:, 1]
        print('Calculating metrics...')
        # Calculate metrics
        print('Calculating auc...')
        auc_score = roc_auc_score(y_test, probs)
        print('Calculating fpr tpr...')
        fpr, tpr, thresholds = roc_curve(y_test, probs)
        ks_stat = max(abs(fpr - tpr))
        print('Calculating hscore...')
        h_measure_score = h_score(y_test, probs) # Assuming h_score is given
        print('Metrics calculating done...')

        # Append scores
        auc_scores.append(auc_score)
        ks_stats.append(ks_stat)
        h_measures.append(h_measure_score)
        print('Appending result done...')
        # Find best cutoff, generate confusion matrix, and calculate losses
        best_cutoff = thresholds[np.argmax(tpr - fpr)]
        y_pred = (probs >= best_cutoff).astype(int)
        cm = confusion_matrix(y_test, y_pred)
        cm_sum += cm
        print('Confusion matrix done...')
        actual_loss = np.sum(X_test[:, loss_col_name] * y_test)
        predicted_loss = np.sum(X_test[:, loss_col_name] * y_pred)
        predicted_loss_in = np.sum(X_test[:, loss_col_name] * y_pred * y_test)
        actual_losses.append(actual_loss)
        predicted_losses.append(predicted_loss)
        predicted_losses_in.append(predicted_loss_in)

        print('Plotting...')
        # Plot Predictive vs Real (for each fold)
        plot_calibration_curve(y_test, probs, ax, label=f'Fold {fold}')
        fold += 1
    plt.savefig(fig_name + '.png', dpi=300)
    # Compute averages of scores and losses
    avg_auc = np.mean(auc_scores)
    avg_ks = np.mean(ks_stats)
    avg_h_measure = np.mean(h_measures)
    avg_actual_loss = np.mean(actual_losses)
    avg_predicted_loss = np.mean(predicted_losses)
    avg_predicted_loss_in = np.mean(predicted_losses_in)
    loss_difference = avg_actual_loss - avg_predicted_loss
    loss_difference_in = avg_actual_loss - avg_predicted_loss_in

    # Print average metrics and loss information
    print(f"Average AUC Score: {avg_auc}")
    print(f"Average KS Statistic: {avg_ks}")
    print(f"Average H-Measure: {avg_h_measure}")
    print(f"Cumulative Confusion Matrix:\n{cm_sum}")
    print(f"Average Actual Loss: {avg_actual_loss}")
    print(f"Average Predicted Loss: {avg_predicted_loss}")
    print(f"Average Predicted Loss: {avg_predicted_loss_in}")
    print(f"Loss Difference: {loss_difference}")
    print(f"Loss Difference: {loss_difference_in}")

    # Return metrics and losses for further use if needed
    return avg_auc, avg_ks, avg_h_measure, cm_sum, avg_actual_loss, avg_predicted_loss, avg_predicted_loss_in, loss_difference, loss_difference_in


In [ ]:
path = '/content/drive/MyDrive/PhD File Sync/GEVGBDT Things/'

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
def evaluate_model_smote(model, X, y, loss_col_name, fig_name):
    tscv = TimeSeriesSplit(n_splits=5)
    fig, ax = plt.subplots(figsize=(12,8))
    auc_scores = []
    ks_stats = []
    h_measures = []
    cm_sum = np.array([[0, 0], [0, 0]]) # Initialize a confusion matrix sum
    actual_losses = []
    predicted_losses = []
    predicted_losses_in = []
    fold = 1
    for train_index, test_index in tscv.split(X):
        print('Fold trial starts...')

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
        print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))
        sm = SMOTE(random_state = 202402)
        X_tr_res, y_tr_res = sm.fit_resample(X_train, y_train)

        print('After OverSampling, the shape of train_X: {}'.format(X_tr_res.shape))
        print('After OverSampling, the shape of train_y: {} \n'.format(y_tr_res.shape))

        print("After OverSampling, counts of label '1': {}".format(sum(y_tr_res==1)))
        print("After OverSampling, counts of label '0': {}".format(sum(y_tr_res==0)))

        model.fit(X_tr_res, y_tr_res)
        probs = model.predict_proba(X_test)[:, 1]
        print('Calculating metrics...')
        # Calculate metrics
        print('Calculating auc...')
        auc_score = roc_auc_score(y_test, probs)
        print('Calculating fpr tpr...')
        fpr, tpr, thresholds = roc_curve(y_test, probs)
        ks_stat = max(abs(fpr - tpr))
        print('Calculating hscore...')
        h_measure_score = h_score(y_test, probs) # Assuming h_score is given
        print('Metrics calculating done...')

        # Append scores
        auc_scores.append(auc_score)
        ks_stats.append(ks_stat)
        h_measures.append(h_measure_score)
        print('Appending result done...')
        # Find best cutoff, generate confusion matrix, and calculate losses
        best_cutoff = thresholds[np.argmax(tpr - fpr)]
        y_pred = (probs >= best_cutoff).astype(int)
        cm = confusion_matrix(y_test, y_pred)
        cm_sum += cm
        print('Confusion matrix done...')
        actual_loss = np.sum(X_test[:, loss_col_name] * y_test)
        predicted_loss = np.sum(X_test[:, loss_col_name] * y_pred)
        predicted_loss_in = np.sum(X_test[:, loss_col_name] * y_pred * y_test)
        actual_losses.append(actual_loss)
        predicted_losses.append(predicted_loss)
        predicted_losses_in.append(predicted_loss_in)

        print('Plotting...')
        # Plot Predictive vs Real (for each fold)
        plot_calibration_curve(y_test, probs, ax, label=f'Fold {fold}')
        fold += 1
    plt.savefig(fig_name+'.png', dpi=300)
    # Compute averages of scores and losses
    avg_auc = np.mean(auc_scores)
    avg_ks = np.mean(ks_stats)
    avg_h_measure = np.mean(h_measures)
    avg_actual_loss = np.mean(actual_losses)
    avg_predicted_loss = np.mean(predicted_losses)
    avg_predicted_loss_in = np.mean(predicted_losses_in)
    loss_difference = avg_actual_loss - avg_predicted_loss
    loss_difference_in = avg_actual_loss - avg_predicted_loss_in

    # Print average metrics and loss information
    print(f"Average AUC Score: {avg_auc}")
    print(f"Average KS Statistic: {avg_ks}")
    print(f"Average H-Measure: {avg_h_measure}")
    print(f"Cumulative Confusion Matrix:\n{cm_sum}")
    print(f"Average Actual Loss: {avg_actual_loss}")
    print(f"Average Predicted Loss: {avg_predicted_loss}")
    print(f"Average Predicted Loss: {avg_predicted_loss_in}")
    print(f"Loss Difference: {loss_difference}")
    print(f"Loss Difference: {loss_difference_in}")

    # Return metrics and losses for further use if needed
    return avg_auc, avg_ks, avg_h_measure, cm_sum, avg_actual_loss, avg_predicted_loss, avg_predicted_loss_in, loss_difference, loss_difference_in


In [ ]:
X_train = pd.read_csv(path + "GvB_train_X.csv")
X_test = pd.read_csv(path + "GvB_test_X.csv")
y_train = pd.read_csv(path + "GvB_train_y.csv", header = None)
y_test = pd.read_csv(path + "GvB_test_y.csv",header = None)

variable_list = ["annuity2principal","dti_x","tot_hi_cred_lim","annual_inc","total_bc_limit",
                 "revol2inc","total_il_high_credit_limit","total_bal_ex_mort","revol_util",
                 "CreditHistoryLength","MONTHLYCONTRACTAMT","income2principal",
                 "revol_bal","total_rev_hi_lim","PBAL_BEG_PERIOD"]

X_train = X_train[variable_list].values
X_test = X_test[variable_list].values

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
X = np.vstack([X_train, X_test])
y = np.vstack([y_train, y_test]).ravel()

In [ ]:
X[:,-1] * y.ravel()

### LC Dataset - Experiment

In [ ]:
lgbm = lgb.LGBMClassifier(boosting_type ='gbdt',
                      class_weight='balanced',
                      learning_rate= 0.01906318880018833,
                      max_depth = 7,
                      num_leaves = 44,
                      objective='binary',
                      n_jobs = -1,
                      metric = 'auc',
                      verbose=-1)

rf = RandomForestClassifier(n_estimators=500, n_jobs=-1, class_weight="balanced", max_depth=7, max_leaf_nodes=44,
                            min_samples_leaf=480)

In [ ]:
evaluate_model(lgbm, X, y, -1, 'lgbm_cv5_calibration')

In [ ]:
evaluate_model(rf, X, y, -1,'rf_cv5_calibration')

In [ ]:
lr = LogisticRegression(solver="sag", max_iter=10000, class_weight="balanced")
evaluate_model(lr, X, y, -1, 'lr_cv5_calibration')

In [ ]:
tau = -0.3
params = {"objective":"binary", "boosting_type":"gbdt", "verbose": -1, "metric": "auc", 'learning_rate': 0.01906318880018833, 'max_depth': -1,  "objective": custom_loss}

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
fig, ax = plt.subplots(figsize=(12, 8))
auc_scores = []
ks_stats = []
h_measures = []
cm_sum = np.array([[0, 0], [0, 0]]) # Initialize a confusion matrix sum
actual_losses = []
predicted_losses = []
predicted_losses_in = []
fold = 1

for train_index, test_index in tscv.split(X):
    print('Fold trial starts...')
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_test = lgb.Dataset(X_test, y_test, reference = lgb_train)

    gbm = lgb.train(params,
                lgb_train,
                valid_sets = lgb_test,
                num_boost_round = 1000,
                callbacks=[lgb.early_stopping(stopping_rounds=50),])

    probs = gbm.predict(X_test)
    probs = transform_gev(probs, -0.3)
    print('Calculating metrics...')
    # Calculate metrics
    print('Calculating auc...')
    auc_score = roc_auc_score(y_test, probs)
    print('Calculating fpr tpr...')
    fpr, tpr, thresholds = roc_curve(y_test, probs)
    ks_stat = max(abs(fpr - tpr))
    print('Calculating hscore...')
    scaler = MinMaxScaler(feature_range=(0, 1))

    # Fit and transform the data
    probs_std = np.clip(probs_std, 0,1)
    h_measure_score = h_score(y_test, probs_std) # Assuming h_score is given
    print('Metrics calculating done...')

    # Append scores
    auc_scores.append(auc_score)
    ks_stats.append(ks_stat)
    h_measures.append(h_measure_score)
    print('Appending result done...')
    # Find best cutoff, generate confusion matrix, and calculate losses
    best_cutoff = thresholds[np.argmax(tpr - fpr)]
    y_pred = (probs >= best_cutoff).astype(int)
    cm = confusion_matrix(y_test, y_pred)
    cm_sum += cm
    print('Confusion matrix done...')
    actual_loss = np.sum(X_test[:, -1] * y_test)
    predicted_loss = np.sum(X_test[:, -1] * y_pred)
    predicted_loss_in = np.sum(X_test[:,-1] * y_pred * y_test)
    actual_losses.append(actual_loss)
    predicted_losses.append(predicted_loss)
    predicted_losses_in.append(predicted_loss_in)
    print('Plotting...')
    # Plot Predictive vs Real (for each fold)
    plot_calibration_curve(y_test, probs_std, ax, label=f'Fold {fold}')
    fold += 1

plt.savefig('gev_gbdt_scaler.png', dpi=300)
# Compute averages of scores and losses
avg_auc = np.mean(auc_scores)
avg_ks = np.mean(ks_stats)
avg_h_measure = np.mean(h_measures)
avg_actual_loss = np.mean(actual_losses)
avg_predicted_loss = np.mean(predicted_losses)
avg_predicted_loss_in = np.mean(predicted_losses_in)
loss_difference = avg_actual_loss - avg_predicted_loss
loss_difference_in = avg_actual_loss - avg_predicted_loss_in
# Print average metrics and loss information
print(f"Average AUC Score: {avg_auc}")
print(f"Average KS Statistic: {avg_ks}")
print(f"Average H-Measure: {avg_h_measure}")
print(f"Cumulative Confusion Matrix:\n{cm_sum}")
print(f"Average Actual Loss: {avg_actual_loss}")
print(f"Average Predicted Loss: {avg_predicted_loss}")
print(f"Average Predicted Loss: {avg_predicted_loss_in}")
print(f"Loss Difference: {loss_difference}")
print(f"Loss Difference: {loss_difference_in}")

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
fig, ax = plt.subplots(figsize=(12,8))
auc_scores = []
ks_stats = []
h_measures = []
cm_sum = np.array([[0, 0], [0, 0]]) # Initialize a confusion matrix sum
actual_losses = []
predicted_losses = []
predicted_losses_in = []
fold = 1

for train_index, test_index in tscv.split(X):
    print('Fold trial starts...')

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_test = lgb.Dataset(X_test, y_test, reference = lgb_train)

    gbm = lgb.train(params,
                lgb_train,
                valid_sets = lgb_test,
                num_boost_round = 1000,
                callbacks=[lgb.early_stopping(stopping_rounds=50),])

    probs = gbm.predict(X_test)
    probs = transform_gev(probs, -0.3)
    print('Calculating metrics...')
    # Calculate metrics
    print('Calculating auc...')
    auc_score = roc_auc_score(y_test, probs)
    print('Calculating fpr tpr...')
    fpr, tpr, thresholds = roc_curve(y_test, probs)
    ks_stat = max(abs(fpr - tpr))
    print('Calculating hscore...')
    scaler = MinMaxScaler(feature_range=(0, 1))
    # Fit and transform the data
    probs_std = scaler.fit_transform(probs.reshape(-1,1))
    probs_std = np.clip(probs_std, 0,1)
    h_measure_score = h_score(y_test, probs) # Assuming h_score is given
    print('Metrics calculating done...')

    # Append scores
    auc_scores.append(auc_score)
    ks_stats.append(ks_stat)
    h_measures.append(h_measure_score)
    print('Appending result done...')
    # Find best cutoff, generate confusion matrix, and calculate losses
    best_cutoff = thresholds[np.argmax(tpr - fpr)]
    y_pred = (probs >= best_cutoff).astype(int)
    cm = confusion_matrix(y_test, y_pred)
    cm_sum += cm
    print('Confusion matrix done...')
    actual_loss = np.sum(X_test[:, -1] * y_test)
    predicted_loss = np.sum(X_test[:, -1] * y_pred)
    predicted_loss_in = np.sum(X_test[:,-1] * y_pred * y_test)
    actual_losses.append(actual_loss)
    predicted_losses.append(predicted_loss)
    predicted_losses_in.append(predicted_loss_in)
    print('Plotting...')
    # Plot Predictive vs Real (for each fold)
    plot_calibration_curve(y_test, probs, ax, label=f'Fold {fold}')
    fold += 1

plt.savefig('gev_gbdt_link.png', dpi=300)
# Compute averages of scores and losses
avg_auc = np.mean(auc_scores)
avg_ks = np.mean(ks_stats)
avg_h_measure = np.mean(h_measures)
avg_actual_loss = np.mean(actual_losses)
avg_predicted_loss = np.mean(predicted_losses)
avg_predicted_loss_in = np.mean(predicted_losses_in)
loss_difference = avg_actual_loss - avg_predicted_loss
loss_difference_in = avg_actual_loss - avg_predicted_loss_in
# Print average metrics and loss information
print(f"Average AUC Score: {avg_auc}")
print(f"Average KS Statistic: {avg_ks}")
print(f"Average H-Measure: {avg_h_measure}")
print(f"Cumulative Confusion Matrix:\n{cm_sum}")
print(f"Average Actual Loss: {avg_actual_loss}")
print(f"Average Predicted Loss: {avg_predicted_loss}")
print(f"Average Predicted Loss in: {avg_predicted_loss_in}")
print(f"Loss Difference: {loss_difference}")
print(f"Loss Difference in: {loss_difference_in}")

In [ ]:
lr = LogisticRegression(solver="sag", max_iter=10000)
evaluate_model_smote(lr, X, y, -1, 'lr_smote_cv5_calibration')

In [ ]:
lgbm = lgb.LGBMClassifier(boosting_type ='gbdt',
                      learning_rate= 0.01906318880018833,
                      max_depth = 7,
                      num_leaves = 44,
                      objective='binary',
                      n_jobs = -1,
                      metric = 'auc',
                      verbose=-1)

rf = RandomForestClassifier(n_estimators=500, n_jobs=-1, max_depth=7, max_leaf_nodes=44,
                            min_samples_leaf=480)

In [ ]:
evaluate_model_smote(rf, X, y, -1, 'rf_smote_cv5_calibration')

In [ ]:
evaluate_model_smote(lgbm, X, y, -1, 'lgbm_smote_cv5_calibration')

## SMF-1

In [ ]:
df_sbf1 = pd.read_csv(path + "sbf_v2.csv")
df_sbf1 = df_sbf1.iloc[:,1:]
Categorical_list = ['The status of bank accounts','Sales range of products', "Educational Background",
                   "Location","Industry","Loan type","Currency Type"]
for i in Categorical_list:
    df_sbf1 = df_sbf1.join(pd.get_dummies(df_sbf1[i], dtype = np.float64))
df_sbf1 = df_sbf1.drop(columns = Categorical_list)
print(df_sbf1.shape)
print(df_sbf1.columns)

In [ ]:
Independent = ['Total Asset',
 'Total Liability',
 'TL/TA',
 'OCF/CL',
 'Quick Ratio',
 'Current Ratio',
 'Cash/Revenue from Operation',
 'EBIT/CL',
 'NCL/(NCL+E)',
 'OCF/TA',
 'Equity Ratio',
 'Acid-test Ratio',
 'OCF/Net Profit',
 'E/(Clending + NCLending)',
 'NFA/E',
 'Cash Ratio',
 '(E+NCL)/(FA+Investment)',
 'Total Unpaid Lending/Equity',
 'Total Unpaid Lending/TA',
 'OCF/TL',
 'EBITDA/TL',
 'ROE',
 'OCF/Revenue',
 'Net Profit Margin on Sales',
 'ROA',
 'Operating Profit Ratio',
 'Net Profit/Total Expense and Cost',
 'Gross Profit Margin',
 'Net Profit/ Cost and Expense',
 'EBITDA',
 'EBITDA/Revenue',
 'Net Profit',
 'OCF',
 'CF from Operation Activities',
 'Receivables Turnover Ratio',
 'Inventory Turnover Ratio',
 'Total Asset Turnover',
 'Current Asset Turnover',
 'Fixed Asset Turnover',
 'Equity Turnover',
 'Working Capital/CA',
 'Rate of Return on Investment',
 'Payable Turnover Ratio',
 'Cash Conversion Cycle',
 'Revenue Growth',
 'Net Profit Growth',
 'TA Growth',
 'Rate of Capital Accumulation',
 'R/E Growth',
 ' Years of employment in relevant industry',
 'Audited or Not',
 'Recognized level of new product',
 'Patent Status',
 'Date of Establishment',
 'Level of famous products',
 'The proportion of total amount of money collected by enterprises through this bank',
 'lending default record of legal representative',
 'Credit card record of legal representative',
 'Marital Status',
 'Residential Status',
 'Year of residence of legal representative',
 'Gender',
 'Age',
 'The value of vehicle and real estates of legal representative',
 'Year of employment in this position',
 'Category of registered capital ',
 'Enterprise credit status in the past three years',
 'Corporate tax record',
 'Legal Issue',
 'Compliance Status',
 'Number of Defaults',
 'Industry Index',
 'Year-end balance of per capita savings of urban and rural residents (yuan/person)',
 'GDP Growth Rate',
 'CPI',
 'Per capita disposable income of urban residents (yuan) / person)',
 'Engel coefficient',
 'Principal',      # 77
 'Amount of Interests and Principals',
 'Score of pledge/collateral',
 'General Account',
 'No account or Missing',
 'Primary Account',
 'Domestically',
 'Internationally',
 'Others or Missing',
 '3yr College',
 'Bachelor+',
 'High School',
 'Middle/Primary School',
 'Others or missing',
 '上海市',
 '北京市',
 '四川成都市',
 '四川省乐山市',
 '四川省凉山州',
 '四川省南充市',
 '四川省广安市',
 '四川省彭州市',
 '四川省攀枝花市',
 '四川省自贡市',
 '四川省资阳市',
 '四川省邛崃市',
 '四川雅安',
 '天津市',
 '江西吉安',
 '河北省唐山市',
 '河南省新乡市',
 '辽宁省丹东市',
 '辽宁省大连市',
 '辽宁省本溪市',
 '辽宁省沈阳市',
 '辽宁省盘锦市',
 '辽宁省营口市',
 '辽宁省铁岭市',
 '辽宁省锦州市',
 '辽宁省鞍山市',
 '重庆市',
 '交通运输业企业',
 '仓储企业',
 '住宿、餐饮业',
 '信息传输业',
 '其他企业',
 '工业企业',
 '建筑企业',
 '房地产开发经营',
 '批发业企业',
 '租赁和商务服务业',
 '软件和信息技术服务业',
 '零售业企业',
 '一般中长期贷款',
 '中长期房地产开发贷款',
 '中长期项目贷款',
 '信用证项下打包贷款',
 '信用证项下进口押汇',
 '出口信用证押汇',
 '出口商业发票融资',
 '商业承兑汇票贴现',
 '应收账款质押贷款',
 '短期流动资金贷款',
 '短期项目贷款',
 '进口代收押汇',
 '银行承兑汇票贴现',
 '人民币',
 '日元',
 '欧元',
 '港币',
 '美元']

In [ ]:
df_sbf1.isnull().any().any()

In [ ]:
X = df_sbf1[Independent].values
y = df_sbf1["Default"].values

In [ ]:
lgbm = lgb.LGBMClassifier(boosting_type ='gbdt',
                      class_weight='balanced',
                      learning_rate= 0.01906318880018833,
                      max_depth = 7,
                      num_leaves = 44,
                      objective='binary',
                      n_jobs = -1,
                      metric = 'auc',
                      verbose=-1)

rf = RandomForestClassifier(n_estimators=500, n_jobs=-1, class_weight="balanced", max_depth=7, max_leaf_nodes=44,
                            min_samples_leaf=480)

#evaluate_model(lgbm, X, y, 77, 'lgbm_smf1_calibration')

In [ ]:
evaluate_model(lgbm, X, y, 77, 'lgbm_smf1_calibration')

In [ ]:
evaluate_model(rf, X, y, 77, 'rf_smf1_calibration')

In [ ]:
lr = LogisticRegression(solver="sag", max_iter=10000, class_weight="balanced")
evaluate_model(lr, X, y, 77, 'lr_smf1_calibration')

In [ ]:
lr = LogisticRegression(solver="sag", max_iter=10000)
evaluate_model_smote(lr, X, y, 77, 'lr_smf1_calibration')

In [ ]:
lgbm = lgb.LGBMClassifier(boosting_type ='gbdt',
                      learning_rate= 0.01906318880018833,
                      max_depth = 7,
                      num_leaves = 44,
                      objective='binary',
                      n_jobs = -1,
                      metric = 'auc',
                      verbose=-1)

rf = RandomForestClassifier(n_estimators=500, n_jobs=-1, max_depth=7, max_leaf_nodes=44,
                            min_samples_leaf=480)

In [ ]:
evaluate_model_smote(rf, X, y, 77, 'rf_smf1_calibration_smote')

In [ ]:
evaluate_model_smote(lgbm, X, y, 77, 'lr_smf1_calibration_smote')

In [ ]:
tau = -0.45
params = {"objective":"binary", "boosting_type":"gbdt", "verbose": -1, "metric": "auc", 'learning_rate': 0.01906318880018833, 'max_depth': -1,  "objective": custom_loss}

tscv = TimeSeriesSplit(n_splits=5)
fig, ax = plt.subplots(figsize=(12,8))
auc_scores = []
ks_stats = []
h_measures = []
cm_sum = np.array([[0, 0], [0, 0]]) # Initialize a confusion matrix sum
actual_losses = []
predicted_losses = []
predicted_losses_in = []
fold = 1

for train_index, test_index in tscv.split(X):
    print('Fold trial starts...')

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_test = lgb.Dataset(X_test, y_test, reference = lgb_train)

    gbm = lgb.train(params,
                lgb_train,
                valid_sets = lgb_test,
                num_boost_round = 1000,
                callbacks=[lgb.early_stopping(stopping_rounds=50),])

    probs = gbm.predict(X_test)
    probs = transform_gev(probs, -0.45)
    print('Calculating metrics...')
    # Calculate metrics
    print('Calculating auc...')
    auc_score = roc_auc_score(y_test, probs)
    print('Calculating fpr tpr...')
    fpr, tpr, thresholds = roc_curve(y_test, probs)
    ks_stat = max(abs(fpr - tpr))
    print('Calculating hscore...')
    scaler = MinMaxScaler(feature_range=(0, 1))
    # Fit and transform the data
    probs_std = scaler.fit_transform(probs.reshape(-1,1))
    probs_std = np.clip(probs_std, 0,1)
    h_measure_score = h_score(y_test, probs) # Assuming h_score is given
    print('Metrics calculating done...')

    # Append scores
    auc_scores.append(auc_score)
    ks_stats.append(ks_stat)
    h_measures.append(h_measure_score)
    print('Appending result done...')
    # Find best cutoff, generate confusion matrix, and calculate losses
    best_cutoff = thresholds[np.argmax(tpr - fpr)]
    y_pred = (probs >= best_cutoff).astype(int)
    cm = confusion_matrix(y_test, y_pred)
    cm_sum += cm
    print('Confusion matrix done...')
    actual_loss = np.sum(X_test[:, 77] * y_test)
    predicted_loss = np.sum(X_test[:, 77] * y_pred)
    predicted_loss_in = np.sum(X_test[:,77] * y_pred * y_test)
    actual_losses.append(actual_loss)
    predicted_losses.append(predicted_loss)
    predicted_losses_in.append(predicted_loss_in)
    print('Plotting...')
    # Plot Predictive vs Real (for each fold)
    plot_calibration_curve(y_test, probs, ax, label=f'Fold {fold}')
    fold += 1

plt.savefig('gev_gbdt_link.png', dpi=300)
# Compute averages of scores and losses
avg_auc = np.mean(auc_scores)
avg_ks = np.mean(ks_stats)
avg_h_measure = np.mean(h_measures)
avg_actual_loss = np.mean(actual_losses)
avg_predicted_loss = np.mean(predicted_losses)
avg_predicted_loss_in = np.mean(predicted_losses_in)
loss_difference = avg_actual_loss - avg_predicted_loss
loss_difference_in = avg_actual_loss - avg_predicted_loss_in
# Print average metrics and loss information
print(f"Average AUC Score: {avg_auc}")
print(f"Average KS Statistic: {avg_ks}")
print(f"Average H-Measure: {avg_h_measure}")
print(f"Cumulative Confusion Matrix:\n{cm_sum}")
print(f"Average Actual Loss: {avg_actual_loss}")
print(f"Average Predicted Loss: {avg_predicted_loss}")
print(f"Average Predicted Loss in: {avg_predicted_loss_in}")
print(f"Loss Difference: {loss_difference}")
print(f"Loss Difference in: {loss_difference_in}")

## SMF-2

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
def evaluate_model_kf(model, X, y, loss_col_name, fig_name):
    tscv = KFold(n_splits=5)
    fig, ax = plt.subplots(figsize=(12,8))
    auc_scores = []
    ks_stats = []
    h_measures = []
    cm_sum = np.array([[0, 0], [0, 0]]) # Initialize a confusion matrix sum
    actual_losses = []
    predicted_losses = []
    predicted_losses_in = []
    fold = 1
    for train_index, test_index in tscv.split(X):
        print('Fold trial starts...')
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train, y_train)
        probs = model.predict_proba(X_test)[:, 1]
        print('Calculating metrics...')
        # Calculate metrics
        print('Calculating auc...')
        auc_score = roc_auc_score(y_test, probs)
        print('Calculating fpr tpr...')
        fpr, tpr, thresholds = roc_curve(y_test, probs)
        ks_stat = max(abs(fpr - tpr))
        print('Calculating hscore...')
        h_measure_score = h_score(y_test, probs) # Assuming h_score is given
        print('Metrics calculating done...')

        # Append scores
        auc_scores.append(auc_score)
        ks_stats.append(ks_stat)
        h_measures.append(h_measure_score)
        print('Appending result done...')
        # Find best cutoff, generate confusion matrix, and calculate losses
        best_cutoff = thresholds[np.argmax(tpr - fpr)]
        y_pred = (probs >= best_cutoff).astype(int)
        cm = confusion_matrix(y_test, y_pred)
        cm_sum += cm
        print('Confusion matrix done...')
        actual_loss = np.sum(X_test[:, loss_col_name] * y_test)
        predicted_loss = np.sum(X_test[:, loss_col_name] * y_pred)
        predicted_loss_in = np.sum(X_test[:, loss_col_name] * y_pred * y_test)
        actual_losses.append(actual_loss)
        predicted_losses.append(predicted_loss)
        predicted_losses_in.append(predicted_loss_in)

        print('Plotting...')
        # Plot Predictive vs Real (for each fold)
        plot_calibration_curve(y_test, probs, ax, label=f'Fold {fold}')
        fold += 1
    plt.savefig(fig_name + '.png', dpi=300)
    # Compute averages of scores and losses
    avg_auc = np.mean(auc_scores)
    avg_ks = np.mean(ks_stats)
    avg_h_measure = np.mean(h_measures)
    avg_actual_loss = np.mean(actual_losses)
    avg_predicted_loss = np.mean(predicted_losses)
    avg_predicted_loss_in = np.mean(predicted_losses_in)
    loss_difference = avg_actual_loss - avg_predicted_loss
    loss_difference_in = avg_actual_loss - avg_predicted_loss_in

    # Print average metrics and loss information
    print(f"Average AUC Score: {avg_auc}")
    print(f"Average KS Statistic: {avg_ks}")
    print(f"Average H-Measure: {avg_h_measure}")
    print(f"Cumulative Confusion Matrix:\n{cm_sum}")
    print(f"Average Actual Loss: {avg_actual_loss}")
    print(f"Average Predicted Loss: {avg_predicted_loss}")
    print(f"Average Predicted Loss: {avg_predicted_loss_in}")
    print(f"Loss Difference: {loss_difference}")
    print(f"Loss Difference: {loss_difference_in}")

    # Return metrics and losses for further use if needed
    return avg_auc, avg_ks, avg_h_measure, cm_sum, avg_actual_loss, avg_predicted_loss, avg_predicted_loss_in, loss_difference, loss_difference_in

def evaluate_model_smote_kf(model, X, y, loss_col_name, fig_name):
    tscv =KFold(n_splits=5)
    fig, ax = plt.subplots(figsize=(12,8))
    auc_scores = []
    ks_stats = []
    h_measures = []
    cm_sum = np.array([[0, 0], [0, 0]]) # Initialize a confusion matrix sum
    actual_losses = []
    predicted_losses = []
    predicted_losses_in = []
    fold = 1
    for train_index, test_index in tscv.split(X):
        print('Fold trial starts...')

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
        print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))
        sm = SMOTE(random_state = 202402)
        X_tr_res, y_tr_res = sm.fit_resample(X_train, y_train)

        print('After OverSampling, the shape of train_X: {}'.format(X_tr_res.shape))
        print('After OverSampling, the shape of train_y: {} \n'.format(y_tr_res.shape))

        print("After OverSampling, counts of label '1': {}".format(sum(y_tr_res==1)))
        print("After OverSampling, counts of label '0': {}".format(sum(y_tr_res==0)))

        model.fit(X_tr_res, y_tr_res)
        probs = model.predict_proba(X_test)[:, 1]
        print('Calculating metrics...')
        # Calculate metrics
        print('Calculating auc...')
        auc_score = roc_auc_score(y_test, probs)
        print('Calculating fpr tpr...')
        fpr, tpr, thresholds = roc_curve(y_test, probs)
        ks_stat = max(abs(fpr - tpr))
        print('Calculating hscore...')
        h_measure_score = h_score(y_test, probs) # Assuming h_score is given
        print('Metrics calculating done...')

        # Append scores
        auc_scores.append(auc_score)
        ks_stats.append(ks_stat)
        h_measures.append(h_measure_score)
        print('Appending result done...')
        # Find best cutoff, generate confusion matrix, and calculate losses
        best_cutoff = thresholds[np.argmax(tpr - fpr)]
        y_pred = (probs >= best_cutoff).astype(int)
        cm = confusion_matrix(y_test, y_pred)
        cm_sum += cm
        print('Confusion matrix done...')
        actual_loss = np.sum(X_test[:, loss_col_name] * y_test)
        predicted_loss = np.sum(X_test[:, loss_col_name] * y_pred)
        predicted_loss_in = np.sum(X_test[:, loss_col_name] * y_pred * y_test)
        actual_losses.append(actual_loss)
        predicted_losses.append(predicted_loss)
        predicted_losses_in.append(predicted_loss_in)

        print('Plotting...')
        # Plot Predictive vs Real (for each fold)
        plot_calibration_curve(y_test, probs, ax, label=f'Fold {fold}')
        fold += 1
    plt.savefig(fig_name+'.png', dpi=300)
    # Compute averages of scores and losses
    avg_auc = np.mean(auc_scores)
    avg_ks = np.mean(ks_stats)
    avg_h_measure = np.mean(h_measures)
    avg_actual_loss = np.mean(actual_losses)
    avg_predicted_loss = np.mean(predicted_losses)
    avg_predicted_loss_in = np.mean(predicted_losses_in)
    loss_difference = avg_actual_loss - avg_predicted_loss
    loss_difference_in = avg_actual_loss - avg_predicted_loss_in

    # Print average metrics and loss information
    print(f"Average AUC Score: {avg_auc}")
    print(f"Average KS Statistic: {avg_ks}")
    print(f"Average H-Measure: {avg_h_measure}")
    print(f"Cumulative Confusion Matrix:\n{cm_sum}")
    print(f"Average Actual Loss: {avg_actual_loss}")
    print(f"Average Predicted Loss: {avg_predicted_loss}")
    print(f"Average Predicted Loss: {avg_predicted_loss_in}")
    print(f"Loss Difference: {loss_difference}")
    print(f"Loss Difference: {loss_difference_in}")

    # Return metrics and losses for further use if needed
    return avg_auc, avg_ks, avg_h_measure, cm_sum, avg_actual_loss, avg_predicted_loss, avg_predicted_loss_in, loss_difference, loss_difference_in

In [ ]:
df_sbf2 = pd.read_excel(path + "sbf_2.xlsx", header = 0)
df_sbf2 = df_sbf2[['Y', 'X1', 'x2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10',
       'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20',
       'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30',
       'X31', 'X32', 'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40',
       'X41', 'X42', 'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50',
       'X51', 'X52', 'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60']]
print(df_sbf2.shape)
print(df_sbf2.columns)

In [ ]:
X = df_sbf2.drop(columns=['Y',"X25", "X23", "X22", "X21", "X29", "X27"]).values
y = df_sbf2['Y'].values

In [ ]:
lgbm = lgb.LGBMClassifier(boosting_type ='gbdt',
                      class_weight='balanced',
                      learning_rate= 0.01906318880018833,
                      max_depth = 7,
                      num_leaves = 44,
                      objective='binary',
                      n_jobs = -1,
                      metric = 'auc',
                      verbose=-1)

rf = RandomForestClassifier(n_estimators=500, n_jobs=-1, class_weight="balanced", max_depth=7, max_leaf_nodes=44,
                            min_samples_leaf=480)

lr = LogisticRegression(solver="sag", max_iter=10000, class_weight="balanced")

In [ ]:
evaluate_model_kf(lgbm, X, y, -1, 'lgbm_smf2_calibration')

In [ ]:
evaluate_model_kf(rf, X, y, -1, 'rf_smf2_calibration')

In [ ]:
evaluate_model_kf(lr, X, y, -1, 'lr_smf2_calibration')

In [ ]:
lgbm = lgb.LGBMClassifier(boosting_type ='gbdt',
                      learning_rate= 0.01906318880018833,
                      max_depth = 7,
                      num_leaves = 44,
                      objective='binary',
                      n_jobs = -1,
                      metric = 'auc',
                      verbose=-1)

rf = RandomForestClassifier(n_estimators=500, n_jobs=-1, max_depth=7, max_leaf_nodes=44,
                            min_samples_leaf=480)

lr = LogisticRegression(solver="sag", max_iter=10000)

In [ ]:
evaluate_model_smote_kf(lgbm, X, y, -1, 'lgbm_smf2_calibration_smote')

In [ ]:
evaluate_model_smote_kf(lr, X, y, -1, 'lr_smf2_calibration_smote')

In [ ]:
evaluate_model_smote_kf(rf, X, y, -1, 'rf_smf2_calibration_smote')

In [ ]:
tau = -0.35
params = {"objective":"binary", "boosting_type":"gbdt", "verbose": -1, "metric": "auc", 'learning_rate': 0.01906318880018833, 'max_depth': -1,  "objective": custom_loss}

tscv = KFold(n_splits=5)
fig, ax = plt.subplots(figsize=(12,8))
auc_scores = []
ks_stats = []
h_measures = []
cm_sum = np.array([[0, 0], [0, 0]]) # Initialize a confusion matrix sum
actual_losses = []
predicted_losses = []
predicted_losses_in = []
fold = 1

for train_index, test_index in tscv.split(X):
    print('Fold trial starts...')

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_test = lgb.Dataset(X_test, y_test, reference = lgb_train)

    gbm = lgb.train(params,
                lgb_train,
                valid_sets = lgb_test,
                num_boost_round = 1000,
                callbacks=[lgb.early_stopping(stopping_rounds=50),])

    probs = gbm.predict(X_test)
    probs = transform_gev(probs, -0.35)
    print('Calculating metrics...')
    # Calculate metrics
    print('Calculating auc...')
    auc_score = roc_auc_score(y_test, probs)
    print('Calculating fpr tpr...')
    fpr, tpr, thresholds = roc_curve(y_test, probs)
    ks_stat = max(abs(fpr - tpr))
    print('Calculating hscore...')
    scaler = MinMaxScaler(feature_range=(0, 1))
    # Fit and transform the data
    probs_std = scaler.fit_transform(probs.reshape(-1,1))
    probs_std = np.clip(probs_std, 0,1)
    h_measure_score = h_score(y_test, probs) # Assuming h_score is given
    print('Metrics calculating done...')

    # Append scores
    auc_scores.append(auc_score)
    ks_stats.append(ks_stat)
    h_measures.append(h_measure_score)
    print('Appending result done...')
    # Find best cutoff, generate confusion matrix, and calculate losses
    best_cutoff = thresholds[np.argmax(tpr - fpr)]
    y_pred = (probs >= best_cutoff).astype(int)
    cm = confusion_matrix(y_test, y_pred)
    cm_sum += cm
    print('Confusion matrix done...')
    actual_loss = np.sum(X_test[:, -1] * y_test)
    predicted_loss = np.sum(X_test[:, -1] * y_pred)
    predicted_loss_in = np.sum(X_test[:,-1] * y_pred * y_test)
    actual_losses.append(actual_loss)
    predicted_losses.append(predicted_loss)
    predicted_losses_in.append(predicted_loss_in)
    print('Plotting...')
    # Plot Predictive vs Real (for each fold)
    plot_calibration_curve(y_test, probs, ax, label=f'Fold {fold}')
    fold += 1

plt.savefig('gev_gbdt_link.png', dpi=300)
# Compute averages of scores and losses
avg_auc = np.mean(auc_scores)
avg_ks = np.mean(ks_stats)
avg_h_measure = np.mean(h_measures)
avg_actual_loss = np.mean(actual_losses)
avg_predicted_loss = np.mean(predicted_losses)
avg_predicted_loss_in = np.mean(predicted_losses_in)
loss_difference = avg_actual_loss - avg_predicted_loss
loss_difference_in = avg_actual_loss - avg_predicted_loss_in
# Print average metrics and loss information
print(f"Average AUC Score: {avg_auc}")
print(f"Average KS Statistic: {avg_ks}")
print(f"Average H-Measure: {avg_h_measure}")
print(f"Cumulative Confusion Matrix:\n{cm_sum}")
print(f"Average Actual Loss: {avg_actual_loss}")
print(f"Average Predicted Loss: {avg_predicted_loss}")
print(f"Average Predicted Loss in: {avg_predicted_loss_in}")
print(f"Loss Difference: {loss_difference}")
print(f"Loss Difference in: {loss_difference_in}")

## SMF-3

In [ ]:
df = pd.read_csv(path + 'sbf_3_clean.csv')

In [ ]:
df.isnull().any().any()

In [ ]:
y = df['是否违约'].values
X = df.drop(columns=['是否违约']).values

In [ ]:
lgbm = lgb.LGBMClassifier(boosting_type ='gbdt',
                      class_weight='balanced',
                      learning_rate= 0.01906318880018833,
                      max_depth = 7,
                      num_leaves = 44,
                      objective='binary',
                      n_jobs = -1,
                      metric = 'auc',
                      verbose=-1)

rf = RandomForestClassifier(n_estimators=500, n_jobs=-1, class_weight="balanced", max_depth=7, max_leaf_nodes=44,
                            min_samples_leaf=480)

lr = LogisticRegression(solver="sag", max_iter=10000, class_weight="balanced")

In [ ]:
evaluate_model_kf(lgbm, X, y, -1, 'lgbm_smf3_cs')

In [ ]:
evaluate_model_kf(lr, X, y, -1, 'lr_smf3_cs')

In [ ]:
evaluate_model_kf(rf, X, y, -1, 'rf_smf3_cs')

In [ ]:
lgbm = lgb.LGBMClassifier(boosting_type ='gbdt',
                      learning_rate= 0.01906318880018833,
                      max_depth = 7,
                      num_leaves = 44,
                      objective='binary',
                      n_jobs = -1,
                      metric = 'auc',
                      verbose=-1)

rf = RandomForestClassifier(n_estimators=500, n_jobs=-1, max_depth=7, max_leaf_nodes=44,
                            min_samples_leaf=480)

lr = LogisticRegression(solver="sag", max_iter=10000)

In [ ]:
evaluate_model_kf(lgbm, X, y, -1, 'lgbm_smf3_smote')

In [ ]:
evaluate_model_kf(lr, X, y, -1, 'lr_smf3_smote')

In [ ]:
evaluate_model_kf(rf, X, y, -1, 'rf_smf3_smote')

In [ ]:
tau = -0.35
params = {"objective":"binary", "boosting_type":"gbdt", "verbose": -1, "metric": "auc", 'learning_rate': 0.01906318880018833, 'max_depth': -1,  "objective": custom_loss}

tscv = KFold(n_splits=5)
fig, ax = plt.subplots(figsize=(12,8))
auc_scores = []
ks_stats = []
h_measures = []
cm_sum = np.array([[0, 0], [0, 0]]) # Initialize a confusion matrix sum
actual_losses = []
predicted_losses = []
predicted_losses_in = []
fold = 1

for train_index, test_index in tscv.split(X):
    print('Fold trial starts...')

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_test = lgb.Dataset(X_test, y_test, reference = lgb_train)

    gbm = lgb.train(params,
                lgb_train,
                valid_sets = lgb_test,
                num_boost_round = 1000,
                callbacks=[lgb.early_stopping(stopping_rounds=50),])

    probs = gbm.predict(X_test)
    probs = transform_gev(probs, -0.35)
    print('Calculating metrics...')
    # Calculate metrics
    print('Calculating auc...')
    auc_score = roc_auc_score(y_test, probs)
    print('Calculating fpr tpr...')
    fpr, tpr, thresholds = roc_curve(y_test, probs)
    ks_stat = max(abs(fpr - tpr))
    print('Calculating hscore...')
    scaler = MinMaxScaler(feature_range=(0, 1))
    # Fit and transform the data
    probs_std = scaler.fit_transform(probs.reshape(-1,1))
    probs_std = np.clip(probs_std, 0,1)
    h_measure_score = h_score(y_test, probs) # Assuming h_score is given
    print('Metrics calculating done...')

    # Append scores
    auc_scores.append(auc_score)
    ks_stats.append(ks_stat)
    h_measures.append(h_measure_score)
    print('Appending result done...')
    # Find best cutoff, generate confusion matrix, and calculate losses
    best_cutoff = thresholds[np.argmax(tpr - fpr)]
    y_pred = (probs >= best_cutoff).astype(int)
    cm = confusion_matrix(y_test, y_pred)
    cm_sum += cm
    print('Confusion matrix done...')
    actual_loss = np.sum(X_test[:, -1] * y_test)
    predicted_loss = np.sum(X_test[:, -1] * y_pred)
    predicted_loss_in = np.sum(X_test[:,-1] * y_pred * y_test)
    actual_losses.append(actual_loss)
    predicted_losses.append(predicted_loss)
    predicted_losses_in.append(predicted_loss_in)
    print('Plotting...')
    # Plot Predictive vs Real (for each fold)
    plot_calibration_curve(y_test, probs, ax, label=f'Fold {fold}')
    fold += 1

plt.savefig('gev_gbdt_link.png', dpi=300)
# Compute averages of scores and losses
avg_auc = np.mean(auc_scores)
avg_ks = np.mean(ks_stats)
avg_h_measure = np.mean(h_measures)
avg_actual_loss = np.mean(actual_losses)
avg_predicted_loss = np.mean(predicted_losses)
avg_predicted_loss_in = np.mean(predicted_losses_in)
loss_difference = avg_actual_loss - avg_predicted_loss
loss_difference_in = avg_actual_loss - avg_predicted_loss_in
# Print average metrics and loss information
print(f"Average AUC Score: {avg_auc}")
print(f"Average KS Statistic: {avg_ks}")
print(f"Average H-Measure: {avg_h_measure}")
print(f"Cumulative Confusion Matrix:\n{cm_sum}")
print(f"Average Actual Loss: {avg_actual_loss}")
print(f"Average Predicted Loss: {avg_predicted_loss}")
print(f"Average Predicted Loss in: {avg_predicted_loss_in}")
print(f"Loss Difference: {loss_difference}")
print(f"Loss Difference in: {loss_difference_in}")

## Economic Benefit Analysis

In [ ]:

models = ['CS-LR', 'CS-RF', 'CS-GBDT', 'SMOTE-LR', 'SMOTE-RF', 'SMOTE-GBDT', 'GEV-GBDT']
actual_losses = np.array([898478589, 898478589, 898478589, 898478589, 898478589, 898478589, 898478589])
predicted_losses = np.array([2191693360, 2141136208, 1977108477, 2237548541, 2041004443, 1978475722, 1924932136])
error_magnitude = actual_losses - predicted_losses
error_corrected = np.array([288488312, 239774684, 264696379, 278838311, 273127308, 253791874, 215355855])

# Plot setup
x = np.arange(len(models))  # the label locations
width = 0.35  # the width of the bars
# Plot setup for comparison
fig, ax = plt.subplots(figsize=(14, 8))

# Bar charts for errors
rects1 = ax.bar(x - width/2, error_magnitude / 1e6, width, label='Actual Loss - Predicted Loss (Millions)', color='skyblue')
rects2 = ax.bar(x + width/2, error_corrected / 1e6, width, label='Actual Loss - Predicted Loss Correct (Millions)', color='lightcoral')

# Labels, title and custom x-axis tick labels, etc.
ax.set_xlabel('Model')
ax.set_ylabel('Error Magnitude (Millions)')
ax.set_title('Error Magnitude Comparison by Model')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()

# Annotate the smaller magnitude for GEV-GBDT
def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(round(height, 2)),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)

fig.tight_layout()

plt.savefig('eco.png', dpi=300)